In [ ]:
import os
import sys

# Clone or pull part
repo_url = "https://github.com/fraco03/6D_pose.git"
repo_dir = "/content/6D_pose"   #Modify here for kaggle
branch = "main"

# Clone if missing
if not os.path.exists(repo_dir):
    !git clone -b {branch} {repo_url}
    print(f"Cloned {repo_url} to {repo_dir}")
else:
    %cd {repo_dir}
    !git fetch origin
    !git checkout {branch}
    !git reset --hard origin/{branch}
    %cd ..
    print(f"Updated {repo_url} to {repo_dir}")

# Add repository to Python path
if repo_dir not in sys.path:
    sys.path.insert(0, repo_dir)

In [ ]:
from google.colab import drive
from utils.load_data import mount_drive

# Mounting part
mount_drive()

dataset_root = "/content/drive/MyDrive/Linemod_preprocessed" #Modify here for kaggle

print("\n✅ Setup complete!")
print(f"📁 Dataset path: {dataset_root}")

In [ ]:
from .src.detection.yolo_utils import calculate_adapted_map50, visualize_bbox, generate_synthetic_dataset, auto_labeled_dataset

In [ ]:
# CELL 3: GENERATE SYNTHETIC TRAINING DATASET WITH COLLAGES
import os
import cv2
import numpy as np
import random
import yaml
import shutil
import requests
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_ROOT = '/content/drive/MyDrive/Linemod_preprocessed'
DEST_ROOT = '/content/datasets'
BG_CACHE_DIR = '/content/backgrounds_cache'

NUM_COLLAGES = 2000      # Number of synthetic collage images to generate
MAX_OBJECTS_PER_IMG = 4  # Maximum objects per collage image

generate_synthetic_dataset(SOURCE_ROOT,DEST_ROOT,BG_CACHE_DIR,NUM_COLLAGES,MAX_OBJECTS_PER_IMG)

In [ ]:
# CELL 4: TRAIN YOLO MODEL ON SYNTHETIC DATA
!pip install ultralytics
from ultralytics import YOLO
import torch

# --- 1. HARDWARE CHECK ---
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ WARNING: Using CPU! Go to Runtime > Change runtime type > T4 GPU")

# --- 2. MODEL SELECTION ---
# YOLO11 Small is perfect for T4 GPU
model = YOLO('yolo11s.pt')

print("🚀 Starting Training on Colab T4...")

# --- 3. TRAINING LOOP ---
results = model.train(
    data='/content/linemod.yaml',

    # --- DURATION ---
    epochs=50,           # Increased to 50. Synthetic data requires more time to converge
    patience=15,         # Early stopping if no improvement for 15 epochs

    # --- HARDWARE ---
    imgsz=640,
    batch=16,            # Batch size 16 is safer and more stable for generalization on difficult datasets
    device=0,
    workers=2,
    amp=True,            # Mixed Precision ON (Essential for T4)
    cache=False,         # Save RAM

    # --- AUGMENTATION (Tuned for "Black-on-Black" scenarios) ---
    mosaic=0.5,          # REDUCED. Data is already collaged, don't overdo it
    mixup=0.0,           # DISABLED. Critical! We want solid objects, not transparent ghosts

    degrees=10.0,        # Light rotation (already done in generator script)
    scale=0.5,           # Zoom in/out
    translate=0.1,       # Light translation
    fliplr=0.5,          # Horizontal flip OK

    # --- LIGHTING (YOLO applies these instead of our script) ---
    hsv_h=0.015,         # Light color shift
    hsv_s=0.7,           # Saturation
    hsv_v=0.4,           # BRIGHTNESS (Value): 0.4 is perfect
                         # Teaches the model: "The phone is the same whether dark (black) or illuminated (gray)"

    close_mosaic=10,     # Last 10 epochs turn off mosaic to refine edges

    # --- SAVE RESULTS ---
    # WARNING: This saves directly to Google Drive
    project='/content/drive/MyDrive/YOLO_Runs',
    name='yolo11_collage_run',
    verbose=True,
    exist_ok=True,
    save=True
)

In [ ]:
# CELL 6: AUTO-LABELING - USE TRAINED MODEL TO ADD MISSING LABELS
import os
import cv2
import numpy as np
import yaml
import shutil
from ultralytics import YOLO
from tqdm import tqdm

# --- CONFIGURATION ---
SOURCE_ROOT = '/content/data/Linemod_preprocessed/data'
DEST_ROOT = '/content/dataset_AUTOLABELED'
MODEL_PATH = '/content/drive/MyDrive/YOLO_Runs/yolo11_collage_run/weights/best.pt'

# Auto-labeling thresholds
CONF_THRESHOLD = 0.80   # Minimum confidence to add predicted label
IOU_THRESHOLD = 0.45    # IoU threshold to detect duplicates with ground truth

auto_labeled_dataset(DEST_ROOT, MODEL_PATH, SOURCE_ROOT, CONF_THRESHOLD, IOU_THRESHOLD )

In [ ]:
# CELL 8: TEST VISUALIZATION FUNCTION
visualize_bbox('/content/dataset_AUTOLABELED/images/train/01_0151.png')

In [ ]:
# CELL 9: FINAL TRAINING ON AUTO-LABELED DATASET
from ultralytics import YOLO

# Use the 's' (Small) clean version
# Will automatically download standard weights if not found
model = YOLO('yolo11s.pt')

results = model.train(
    # The clean dataset with automatic labels
    data='/content/dataset_AUTOLABELED/data.yaml',

    # --- DURATION ---
    epochs=20,
    patience=15,

    # --- HARDWARE ---
    imgsz=640,
    batch=32,            # With the 's' model, use 16 to avoid running out of video memory (OOM)
    device=0,
    workers=4,
    
    # --- AUGMENTATION ---
    mosaic=1.0,          # Now that labels are good, mosaic is useful
    mixup=0.1,           # Light mixup

    degrees=10.0,
    fliplr=0.5,
    scale=0.5,

    # --- OUTPUT ---
    project='/content/drive/MyDrive/YOLO_Runs',
    name='yolo11s_autolabel_final', # Changed name to remind it's the S model
    save=True,
    exist_ok=True
)

In [ ]:
calculate_adapted_map50('/content/drive/MyDrive/YOLO_Runs/yolo11s_autolabel_final/weights/best.pt',
                        '/content/drive/MyDrive/Linemod_preprocessed')